In [3]:
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import scipy
import sklearn
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [96]:
# dataset_red=pd.read_csv("Data/红酒白酒分类+回归（都可以）/winequality-red.csv",sep=";",index_col=False)
# dataset_white=pd.read_csv("Data/红酒白酒分类+回归（都可以）/winequality-white.csv",sep=";",index_col=False)
dataset = pd.read_csv("Data/加速度计 regression/accelerometer.csv")
dataset.head()

,wconfid,pctid,x,y,z
0,1,20,1.004,0.090,-0.125
1,1,20,1.004,-0.043,-0.125
2,1,20,0.969,0.090,-0.121
3,1,20,0.973,-0.012,-0.137
4,1,20,1.000,-0.016,-0.121


In [97]:
#preprocessing
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(dataset["wconfid"])
X = dataset.drop("wconfid", axis=1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=3)
pd.value_counts(y_test)

0    15314
1    15303
2    15283
dtype: int64

In [98]:
#normalization
from sklearn.preprocessing import Normalizer
norm = Normalizer()
Xn_train = norm.fit_transform(X_train)
Xn_test = norm.transform(X_test)
Xn_test.shape

(45900, 4)

In [99]:
#NN model
model = keras.models.Sequential()
model.add(keras.layers.Dense(256, input_dim=4, activation="linear"))
model.add(keras.layers.Dense(256, activation="linear"))
model.add(keras.layers.Dense(256, activation="linear"))
model.add(keras.layers.Dense(256, activation="linear"))
model.add(keras.layers.Dense(3, activation="softmax"))
model.summary()
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_75 (Dense)             (None, 256)               1280      
_________________________________________________________________
dense_76 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_77 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_78 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_79 (Dense)             (None, 3)                 771       
Total params: 199,427
Trainable params: 199,427
Non-trainable params: 0
_________________________________________________________________


In [100]:
#train the model
model.fit(np.array(Xn_train), np.array(y_train), epochs=1000, validation_data=(np.array(Xn_test), np.array(y_test)))

Train on 107100 samples, validate on 45900 samples
Epoch 1/1000
107100/107100 [==============================] - 10s 98us/sample - loss: 1.0997 - acc: 0.3326 - val_loss: 1.0986 - val_acc: 0.3336
Epoch 2/1000
107100/107100 [==============================] - 10s 94us/sample - loss: 1.0989 - acc: 0.3326 - val_loss: 1.0987 - val_acc: 0.3330
Epoch 3/1000
107100/107100 [==============================] - 10s 98us/sample - loss: 1.0989 - acc: 0.3341 - val_loss: 1.0987 - val_acc: 0.3334
Epoch 4/1000
107100/107100 [==============================] - 11s 101us/sample - loss: 1.0991 - acc: 0.3335 - val_loss: 1.0987 - val_acc: 0.3334
Epoch 5/1000
107100/107100 [==============================] - 11s 105us/sample - loss: 1.0989 - acc: 0.3336 - val_loss: 1.0987 - val_acc: 0.3334
Epoch 6/1000
107100/107100 [==============================] - 11s 105us/sample - loss: 1.0990 - acc: 0.3351 - val_loss: 1.0987 - val_acc: 0.3330
Epoch 7/1000
107100/107100 [==============================] - 12s 107us/sample - l

KeyboardInterrupt: 

In [101]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = model.predict_classes(Xn_test)
print(y_pred)
print(y_test)
y_test = np.array(y_test)
y_test = y_test.reshape(y_test.shape[0], 1)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[0 1 1 ... 0 1 1]
[1 0 2 ... 0 1 0]
[[ 2243 13071     0]
 [ 2032 13271     0]
 [    0 15283     0]]


0.33799564270152505

In [64]:
scores = model.evaluate(np.array(Xn_test), np.array(y_test), verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 31.56%


In [88]:
dataset_red=pd.read_csv("Data/红酒白酒分类+回归（都可以）/winequality-red.csv",sep=";",index_col=False)
dataset_white=pd.read_csv("Data/红酒白酒分类+回归（都可以）/winequality-white.csv",sep=";",index_col=False)
# dataset_red.head(n=len(dataset_red))
red=["red"]*len(dataset_red)
dataset_red.insert(0,"Label",red)
white=["white"]*len(dataset_white)
dataset_white.insert(0,"Label",white)
# dataset_white.head(n=5)
dataset=pd.concat([dataset_red,dataset_white],axis=0)
dataset.set_index("Label")

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
Label,,,,,,,,,,,,
red,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
red,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
red,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
red,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
red,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
white,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
white,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
white,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6


In [89]:
#preprocessing
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
dataset["Label"]=le.fit_transform(dataset["Label"])
y = dataset["Label"]
X = dataset.drop('Label',axis=1)

#split the dataset into trainning set and testing set
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=2)
# dataset['Label'].value_counts(normalize=True)
X
y_test.value_counts()

1    1457
0     493
Name: Label, dtype: int64

In [90]:
#Normalization
from sklearn.preprocessing import Normalizer
norm = Normalizer()
norm = norm.fit(X_train)
Xn_train = norm.transform(X_train)
Xn_test = norm.transform(X_test)
Xn_train

array([[0.03929095, 0.00104005, 0.00173342, ..., 0.0030046 , 0.05200273,
        0.04044657],
       [0.03270101, 0.00206533, 0.00120477, ..., 0.00361432, 0.0573702 ,
        0.03442212],
       [0.03880164, 0.00211127, 0.00291012, ..., 0.00251069, 0.05021388,
        0.02853061],
       ...,
       [0.0646795 , 0.00207898, 0.00358047, ..., 0.00669895, 0.1166541 ,
        0.06929947],
       [0.0538591 , 0.0023417 , 0.00351255, ..., 0.00366866, 0.08664289,
        0.05463966],
       [0.04715094, 0.00092873, 0.00214322, ..., 0.00321484, 0.07358404,
        0.04286449]])

In [91]:
#Feature selection
from sklearn.feature_selection import SelectKBest, chi2
selector = SelectKBest(chi2,k=8)
Xnf_train = selector.fit_transform(Xn_train,y_train)
Xnf_test = selector.transform(Xn_test)
Xnf_train

array([[3.92909538e-02, 1.04005466e-03, 9.88051926e-01, ...,
        3.00460235e-03, 5.20027330e-02, 4.04465701e-02],
       [3.27010135e-02, 2.06532717e-03, 9.52345305e-01, ...,
        3.61432254e-03, 5.73701991e-02, 3.44221195e-02],
       [3.88016352e-02, 2.11126544e-03, 9.30098020e-01, ...,
        2.51069404e-03, 5.02138808e-02, 2.85306141e-02],
       ...,
       [6.46795021e-02, 2.07898400e-03, 9.70192532e-01, ...,
        6.69894843e-03, 1.16654102e-01, 6.92994665e-02],
       [5.38590961e-02, 2.34169983e-03, 9.52291265e-01, ...,
        3.66866307e-03, 8.66428938e-02, 5.46396627e-02],
       [4.71509383e-02, 9.28730604e-04, 9.35874685e-01, ...,
        3.21483670e-03, 7.35840401e-02, 4.28644894e-02]])

In [92]:
#defining ANN model with two hidden layers, first hidden layer has 20 neurons, the next one has 10 neurons
model=keras.models.Sequential()
model.add(keras.layers.Dense(256, input_dim=8,activation="linear"))
model.add(keras.layers.Dense(512, activation="linear"))
model.add(keras.layers.Dense(512, activation="linear"))
model.add(keras.layers.Dense(1,activation='sigmoid'))
# model.add(keras.layers.Dense(1, activation='softmax'))

model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_71 (Dense)             (None, 256)               2304      
_________________________________________________________________
dense_72 (Dense)             (None, 512)               131584    
_________________________________________________________________
dense_73 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_74 (Dense)             (None, 1)                 513       
Total params: 397,057
Trainable params: 397,057
Non-trainable params: 0
_________________________________________________________________


In [93]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [94]:
#train the model
history = model.fit((np.array(Xnf_train), np.array(y_train)), epochs=500, validation_data=(np.array(Xnf_test), np.array(y_test)))

Train on 4547 samples, validate on 1950 samples
Epoch 1/500
4547/4547 [==============================] - 1s 245us/sample - loss: 0.0059 - acc: 1.0000 - val_loss: 6.4776 - val_acc: 0.7472
Epoch 2/500
4547/4547 [==============================] - 1s 144us/sample - loss: 7.3210e-11 - acc: 1.0000 - val_loss: 6.4776 - val_acc: 0.7472
Epoch 3/500
4547/4547 [==============================] - 1s 134us/sample - loss: 7.3210e-11 - acc: 1.0000 - val_loss: 6.4776 - val_acc: 0.7472
Epoch 4/500
4547/4547 [==============================] - 1s 140us/sample - loss: 7.3210e-11 - acc: 1.0000 - val_loss: 6.4776 - val_acc: 0.7472
Epoch 5/500
4547/4547 [==============================] - 1s 138us/sample - loss: 7.3209e-11 - acc: 1.0000 - val_loss: 6.4776 - val_acc: 0.7472
Epoch 6/500
4547/4547 [==============================] - 1s 141us/sample - loss: 7.3208e-11 - acc: 1.0000 - val_loss: 6.4776 - val_acc: 0.7472
Epoch 7/500
4547/4547 [==============================] - 1s 144us/sample - loss: 7.3208e-11 - acc:

In [95]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = model.predict_classes(Xnf_test)
y_test = np.array(y_test)
print(y_pred)
print(y_test)
y_test = y_test.reshape((np.array(y_test)).shape[0], 1)
cm = confusion_matrix(np.array(y_test), y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1]
 [1]
 [1]
 ...
 [1]
 [1]
 [1]]
[0 0 1 ... 1 1 0]
[[   0  493]
 [   0 1457]]


0.7471794871794872

In [ ]:
scores = model.evaluate(np.array(Xn_test), np.array(y_test), verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))